# Azure Cosmos DB (Gremlin) — Product Recommendation Graph 

## Objective
This notebook evaluates a **graph-based recommendation system** for an online shop using  
**Azure Cosmos DB (Gremlin API)**.

We will measure:
1. **Graph build time** for increasing data volumes.
2. **Execution time of core recommendation queries**.


Instead of writing everything inside the notebook, we reuse the project modules:
- `gremlin_client.py` → connection + safe query execution + styled logs.
- `data_generator.py` → scalable graph generation based on volume `N`.

This approach ensures:
- maintainability,
- reproducibility,
- clean separation of responsibilities.

## Benchmark volumes
We will test three realistic scenarios:
- **5000 products**
- **1000 products**
- **2000 products**

Each test runs independently in its own cell for clarity.


## 1. Environment Setup

We import standard libraries for timing and reporting, plus project modules.

We also silence verbose Gremlin driver logs to keep outputs readable.


In [1]:
import os, sys
import time
import random
import logging

import pandas as pd
import matplotlib.pyplot as plt

import nest_asyncio
nest_asyncio.apply()

# Silence noisy gremlin driver logs (409 conflicts, etc.)
logging.getLogger("gremlin_python.driver").setLevel(logging.CRITICAL)

# Ensure current folder is on Python path so that "import ..." works
if os.getcwd() not in sys.path:
    sys.path.append(os.getcwd())

from gremlin_client import (
    create_gremlin_client,
    close_gremlin_client,
    run_gremlin,
    print_banner,
    print_step,
    print_ok,
    print_warning,
)

from data_generator import build_graph

%matplotlib inline


## 2. Create a Shared Gremlin Client

We create one client instance that will be reused throughout the notebook.

This mimics a production-style setup where connection logic is centralized.


In [2]:
gclient = create_gremlin_client()



=== Connected to Azure Cosmos DB (Gremlin API) ===


## 3. Benchmark Helpers

We define reusable helper functions to:
- build the graph with timing,
- generate the standard recommendation query set,
- measure query execution time.

These helpers are volume-agnostic and will be reused in each experiment cell.


In [ ]:
# Role:
#   Construire le graphe pour un volume N et mesurer le temps total de construction.
# Inputs:
#   - gclient: client Gremlin connecté
#   - N (int): volume (nombre total de produits souhaité)
# Output:
#   - dict: statistiques de build + temps (build_time_sec) + volume (volume)
def build_graph_with_timing(gclient, N: int) -> dict:
    N = max(10, N)

    print_banner(f"Build graph for N={N}")
    start = time.time()

    stats = build_graph(gclient, N)

    elapsed = time.time() - start

    print_ok(
        f"Graph build for N={N} finished in {elapsed:.2f} seconds "
        f"(run_pk={stats['run_pk']}, {stats['products']} products, {stats['users']} users)."
    )

    out = stats.copy()
    out["build_time_sec"] = elapsed
    out["volume"] = N
    return out


# Role:
#   Exécuter une requête Gremlin, mesurer le temps d’exécution et compter les résultats.
# Inputs:
#   - gclient: client Gremlin connecté
#   - name (str): nom lisible de la requête (pour logs / DataFrame)
#   - query (str): requête Gremlin (chaîne exécutable)
# Output:
#   - dict: ligne de mesure {"query", "time_sec", "num_results"}
def measure_query(gclient, name: str, query: str) -> dict:
    start = time.time()
    results = run_gremlin(gclient, query)
    elapsed = time.time() - start

    num_results = len(results)
    print(f"{name:30s} | {elapsed:.4f} sec | {num_results} results")

    return {"query": name, "time_sec": elapsed, "num_results": num_results}


# Role:
#   Construire une requête Gremlin “chaînée” à partir d’une liste d’étapes, et fournir
#   en même temps une version formatée lisible pour l’affichage.
# Inputs:
#   - steps (List[str]): étapes sans le point (ex: ["g.V(...)", "out('X')", "dedup()"])
# Output:
#   - Tuple[str, str]: (query_exec, query_pretty)
def _build_query_from_steps(steps: List[str]) -> Tuple[str, str]:
    query_exec = ".".join(steps)
    query_pretty = steps[0] + "\n  ." + "\n  .".join(steps[1:])
    return query_exec, query_pretty


# Role:
#   Préparer le set de requêtes de recommandation (4 requêtes) pour un product/user
#   en utilisant le “point-read” Cosmos: g.V([run_pk, id]).
# Inputs:
#   - product_id (str): id du produit cible (ex: "p1")
#   - user_id (str): id utilisateur (ex: "u1")
#   - run_pk (str): valeur de partition key (retournée par build_graph)
# Output:
#   - List[Tuple[str, str, str]]: [(name, query_exec, query_pretty), ...]
def get_query_set(product_id: str, user_id: str, run_pk: str) -> List[Tuple[str, str, str]]:
    queries: List[Tuple[str, str, str]] = []

    # 1) Produits similaires via catégorie (IN_CATEGORY)
    steps_1 = [
        f"g.V(['{run_pk}','{product_id}'])",
        "out('IN_CATEGORY')",
        "in('IN_CATEGORY')",
        "hasLabel('product')",
        "dedup()",
        "limit(20)",
    ]
    q1_exec, q1_pretty = _build_query_from_steps(steps_1)
    queries.append(("Similar_by_Category", q1_exec, q1_pretty))

    # 2) Produits similaires via arêtes SIMILAR_TO (tri par score décroissant)
    steps_2 = [
        f"g.V(['{run_pk}','{product_id}'])",
        "outE('SIMILAR_TO')",
        "order()",
        "by('score', decr)",
        "inV()",
        "dedup()",
        "limit(20)",
    ]
    q2_exec, q2_pretty = _build_query_from_steps(steps_2)
    queries.append(("Similar_by_SIMILAR_TO", q2_exec, q2_pretty))

    # 3) “Customers also bought” (utilisateurs qui ont acheté ce produit -> autres achats)
    steps_3 = [
        f"g.V(['{run_pk}','{product_id}'])",
        "in('BOUGHT')",
        "out('BOUGHT')",
        "hasLabel('product')",
        "dedup()",
        "limit(20)",
    ]
    q3_exec, q3_pretty = _build_query_from_steps(steps_3)
    queries.append(("Customers_Also_Bought", q3_exec, q3_pretty))

    # 4) Reco user-based (produits achetés par des users “connectés” via BOUGHT)
    steps_4 = [
        f"g.V(['{run_pk}','{user_id}'])",
        "out('BOUGHT')",
        "in('BOUGHT')",
        "out('BOUGHT')",
        "hasLabel('product')",
        "dedup()",
        "limit(20)",
    ]
    q4_exec, q4_pretty = _build_query_from_steps(steps_4)
    queries.append(("User_Recommendations", q4_exec, q4_pretty))

    return queries


# Role:
#   Lancer une expérience complète:
#     1) construire le graphe pour un volume N (avec timing)
#     2) exécuter 4 requêtes de référence (avec timing)
#     3) retourner un DataFrame exploitable (build + queries)
# Inputs:
#   - gclient: client Gremlin connecté
#   - N (int): volume de produits
#   - product_id (str): produit de test (défaut "p1")
#   - user_id (str): user de test (défaut "u1")
# Output:
#   - pd.DataFrame: lignes de mesures (phase build + phase query)
def run_volume_experiment(gclient, N: int, product_id: str = "p1", user_id: str = "u1") -> pd.DataFrame:
    rows: List[Dict] = []

    stats = build_graph_with_timing(gclient, N)
    run_pk = stats["run_pk"]

    rows.append(
        {
            "volume": N,
            "run_pk": run_pk,
            "phase": "build_graph",
            "query": "-",
            "time_sec": stats["build_time_sec"],
            "num_results": stats["products"],
        }
    )

    print_step(f"Running recommendation queries for run_pk={run_pk}, product={product_id}, user={user_id}")
    queries = get_query_set(product_id, user_id, run_pk)

    print_banner("Queries used (formatted)")
    for name, _, pretty in queries:
        print(f"\n{name}\n{pretty}\n")

    for name, q_exec, _ in queries:
        row = measure_query(gclient, name, q_exec)
        row["volume"] = N
        row["run_pk"] = run_pk
        row["phase"] = "query"
        rows.append(row)

    return pd.DataFrame(rows)


## 4. Experiment A — Small Volume (500 products)

This test validates:
- correct graph generation at small scale,
- baseline query times.

Expected outcome:
- fast build time,
- stable query execution with non-empty results.


In [4]:
dfs_500 = run_volume_experiment(gclient, 500, product_id="p1", user_id="u1")
print("\n✅ Results for N=500")
display(dfs_500)



=== Build graph for N=500 ===
→ Resetting graph partition for run_pk='bench_V500' ...
✔ Partition cleared
→ Preparing master data for N=500 products...
→ Using 100 users, about 5 interactions per user.
→ Creating vertices (categories, brands, tags, products, users)...


Creating user vertices: 100%|██████████| 100/100 [00:08<00:00, 12.04user/s]


✔ All vertices inserted
→ Creating base relations (IN_CATEGORY, HAS_BRAND, HAS_TAG, PARENT_OF)...


Linking category hierarchy: 100%|██████████| 8/8 [00:00<00:00, 30.89cat/s]


✔ Base relations created
→ Generating user interactions (VIEWED, BOUGHT, LIKED)...


Creating user interactions: 100%|██████████| 100/100 [01:49<00:00,  1.10s/user]


✔ User interactions created
→ Creating advanced edges (SIMILAR_TO, BOUGHT_TOGETHER, SIMILAR_USER)...


Creating SIMILAR_USER edges: 100%|██████████| 100/100 [00:26<00:00,  3.84user/s]


✔ Advanced edges created
✔ Graph build for N=500 finished in 782.37 seconds (run_pk=bench_V500, 500 products, 100 users).
→ Running recommendation queries for run_pk=bench_V500, product=p1, user=u1
Similar_by_Category            | 0.1789 sec | 20 results
Similar_by_SIMILAR_TO          | 0.1312 sec | 3 results
Customers_Also_Bought          | 0.1816 sec | 2 results
User_Recommendations           | 0.2274 sec | 13 results

✅ Results for N=500


,volume,run_pk,phase,query,time_sec,num_results
0,500,bench_V500,build_graph,-,782.370842,500
1,500,bench_V500,query,Similar_by_Category,0.178910,20
2,500,bench_V500,query,Similar_by_SIMILAR_TO,0.131233,3
3,500,bench_V500,query,Customers_Also_Bought,0.181574,2
4,500,bench_V500,query,User_Recommendations,0.227359,13


## 5. Experiment B — Medium Volume (1000 products)

This test evaluates scalability under a more realistic catalog size.

We expect:
- noticeable increase in build time,
- moderate increase in recommendation query latency.


In [5]:
dfs_1000 = run_volume_experiment(gclient, 1000, product_id="p1", user_id="u1")
print("\n✅ Results for N=1000")
display(dfs_1000)



=== Build graph for N=1000 ===
→ Resetting graph partition for run_pk='bench_V1000' ...
✔ Partition cleared
→ Preparing master data for N=1000 products...
→ Using 200 users, about 5 interactions per user.
→ Creating vertices (categories, brands, tags, products, users)...


Creating user vertices: 100%|██████████| 200/200 [00:17<00:00, 11.69user/s]


✔ All vertices inserted
→ Creating base relations (IN_CATEGORY, HAS_BRAND, HAS_TAG, PARENT_OF)...


Linking category hierarchy: 100%|██████████| 8/8 [00:00<00:00, 29.69cat/s]


✔ Base relations created
→ Generating user interactions (VIEWED, BOUGHT, LIKED)...


Creating user interactions: 100%|██████████| 200/200 [03:36<00:00,  1.08s/user]


✔ User interactions created
→ Creating advanced edges (SIMILAR_TO, BOUGHT_TOGETHER, SIMILAR_USER)...


Creating SIMILAR_USER edges: 100%|██████████| 200/200 [00:52<00:00,  3.78user/s]


✔ Advanced edges created
✔ Graph build for N=1000 finished in 1556.15 seconds (run_pk=bench_V1000, 1000 products, 200 users).
→ Running recommendation queries for run_pk=bench_V1000, product=p1, user=u1
Similar_by_Category            | 0.2103 sec | 20 results
Similar_by_SIMILAR_TO          | 0.1335 sec | 3 results
Customers_Also_Bought          | 0.1071 sec | 0 results
User_Recommendations           | 0.2261 sec | 4 results

✅ Results for N=1000


,volume,run_pk,phase,query,time_sec,num_results
0,1000,bench_V1000,build_graph,-,1556.154696,1000
1,1000,bench_V1000,query,Similar_by_Category,0.210301,20
2,1000,bench_V1000,query,Similar_by_SIMILAR_TO,0.133466,3
3,1000,bench_V1000,query,Customers_Also_Bought,0.107144,0
4,1000,bench_V1000,query,User_Recommendations,0.226086,4


## 6. Experiment C — Large Volume (2000 products)

This test stresses the system while still remaining feasible for a student project.

We expect:
- significantly higher build time,
- higher variability in query latency depending on Cosmos throughput and RU limits.


In [6]:
dfs_2000 = run_volume_experiment(gclient, 2000, product_id="p1", user_id="u1")
print("\n✅ Results for N=2000")
display(dfs_2000)



=== Build graph for N=2000 ===
→ Resetting graph partition for run_pk='bench_V2000' ...
✔ Partition cleared
→ Preparing master data for N=2000 products...
→ Using 400 users, about 5 interactions per user.
→ Creating vertices (categories, brands, tags, products, users)...


Creating user vertices: 100%|██████████| 400/400 [00:32<00:00, 12.27user/s]


✔ All vertices inserted
→ Creating base relations (IN_CATEGORY, HAS_BRAND, HAS_TAG, PARENT_OF)...


Linking category hierarchy: 100%|██████████| 8/8 [00:00<00:00, 31.07cat/s]


✔ Base relations created
→ Generating user interactions (VIEWED, BOUGHT, LIKED)...


Creating user interactions: 100%|██████████| 400/400 [07:02<00:00,  1.06s/user]


✔ User interactions created
→ Creating advanced edges (SIMILAR_TO, BOUGHT_TOGETHER, SIMILAR_USER)...


Creating SIMILAR_USER edges: 100%|██████████| 400/400 [01:44<00:00,  3.85user/s]


✔ Advanced edges created
✔ Graph build for N=2000 finished in 3076.56 seconds (run_pk=bench_V2000, 2000 products, 400 users).
→ Running recommendation queries for run_pk=bench_V2000, product=p1, user=u1
Similar_by_Category            | 0.2362 sec | 20 results
Similar_by_SIMILAR_TO          | 0.1326 sec | 3 results
Customers_Also_Bought          | 0.1057 sec | 0 results
User_Recommendations           | 0.2177 sec | 5 results

✅ Results for N=2000


,volume,run_pk,phase,query,time_sec,num_results
0,2000,bench_V2000,build_graph,-,3076.562585,2000
1,2000,bench_V2000,query,Similar_by_Category,0.236241,20
2,2000,bench_V2000,query,Similar_by_SIMILAR_TO,0.132640,3
3,2000,bench_V2000,query,Customers_Also_Bought,0.105716,0
4,2000,bench_V2000,query,User_Recommendations,0.217735,5


## 7. Consolidated Benchmark Report

We merge all results into one table and generate a comparison pivot.

This final summary is the most useful part for your report and presentation.


In [7]:


df_results = pd.concat([dfs_500, dfs_1000, dfs_2000], ignore_index=True)  # ✅ noms cohérents

print("\n✅ Consolidated raw results")
display(df_results)

# --- Query-only comparison (pivot) ---
df_queries = df_results[df_results["phase"] == "query"].copy()

pivot_queries = df_queries.pivot_table(
    index="query",
    columns="volume",
    values="time_sec",
    aggfunc="mean"
).sort_index(axis=1)  # volumes en ordre croissant

print("\n✅ Query time comparison (seconds)")
display(pivot_queries)

# --- Build time summary (optionnel mais utile) ---
df_build = df_results[df_results["phase"] == "build_graph"].copy()

build_summary = df_build.pivot_table(
    index="phase",
    columns="volume",
    values="time_sec",
    aggfunc="mean"
).sort_index(axis=1)

print("\n✅ Build time comparison (seconds)")
display(build_summary)



✅ Consolidated raw results


,volume,run_pk,phase,query,time_sec,num_results
0,500,bench_V500,build_graph,-,782.370842,500
1,500,bench_V500,query,Similar_by_Category,0.178910,20
2,500,bench_V500,query,Similar_by_SIMILAR_TO,0.131233,3
3,500,bench_V500,query,Customers_Also_Bought,0.181574,2
4,500,bench_V500,query,User_Recommendations,0.227359,13
5,1000,bench_V1000,build_graph,-,1556.154696,1000
6,1000,bench_V1000,query,Similar_by_Category,0.210301,20
7,1000,bench_V1000,query,Similar_by_SIMILAR_TO,0.133466,3
8,1000,bench_V1000,query,Customers_Also_Bought,0.107144,0
9,1000,bench_V1000,query,User_Recommendations,0.226086,4



✅ Query time comparison (seconds)


volume,500,1000,2000
query,,,
Customers_Also_Bought,0.181574,0.107144,0.105716
Similar_by_Category,0.178910,0.210301,0.236241
Similar_by_SIMILAR_TO,0.131233,0.133466,0.132640
User_Recommendations,0.227359,0.226086,0.217735



✅ Build time comparison (seconds)


volume,500,1000,2000
phase,,,
build_graph,782.370842,1556.154696,3076.562585
